## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [1]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.3 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [15]:
import numpy as np
import pandas as pd
import numba

recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
recipes = pd.DataFrame(recipes)
reviews = pd.DataFrame(reviews)
recipes[:3]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0


In [3]:
reviews[:3]

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


In [4]:
import time
lst = []
counter = 0
start = time.time()
for index, row in reviews.iterrows():
  if '2010' in row['date']:   
    lst.append(row['rating'])
    counter = counter + row['rating']
end = time.time()
print(counter/len(lst))
print('Время выполнения ',end-start)

4.4544402182900615
Время выполнения  5.3749635219573975


In [ ]:
a = []
for index, row in reviews.iterrows():
  if '2010' in row['date']:
    a.append(row['date'])
df = pd.DataFrame({'rating':lst,'date': a})
start = time.time()
count = 0
lst2 = []
for index, row in df.iterrows():
  count += row['rating']
  lst2.append(row['rating'])
end = time.time()
print(count/len(lst2))
print('Время выполнения ',end-start)

4.4544402182900615
Время выполнения  0.3945624828338623


In [ ]:
start = time.time()
print(df['rating'].mean(axis=0))
end = time.time()
print('Время выполнения ',end-start)

4.4544402182900615
Время выполнения  0.001608133316040039


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [6]:
%load_ext line_profiler

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [8]:
start = time.time()
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
end = time.time()
print("Время выполнения ",end-start)

Время выполнения  0.0005259513854980469


**Ответ:** Функцию можно сократить в количестве выполняемых операций, из-за чего уменьшится и количество переменных, что не будут занимать память и замедлять скорость выполнения (к примеру, применить .split в for, а не создавать отдельную переменную для этого)

In [9]:
start = time.time()
def get_word_reviews_count_new(df):
  word_reviews = {}
  for row in df.dropna(subset=['review'])['review'].str.split(' '):
    for word in row:
      if word in word_reviews:
        word_reviews[word] += 1
  return word_reviews
end = time.time()
print('Время выполнения ',end-start)     


Время выполнения  0.00019669532775878906


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [11]:
#1
def Mape1(rev: pd.DataFrame):
  rating_cl = rev[rev['rating'] != 0]['rating']
  m = rating_cl.mean()
  return rating_cl.apply(lambda x: abs(m - x)).sum()/m*100/rating_cl.count()

%time Mape1(reviews)

CPU times: user 100 ms, sys: 5.88 ms, total: 106 ms
Wall time: 114 ms


11.079150232267242

In [ ]:
#2

In [13]:
#3
def Mape3(rev: pd.DataFrame):
  rating_cl = rev[rev['rating'] != 0]['rating'].to_numpy()
  m = rating_cl.mean()
  return np.absolute(rating_cl - m).sum()/m*100/rating_cl.shape[0]

%time Mape3(reviews)

CPU times: user 21.6 ms, sys: 0 ns, total: 21.6 ms
Wall time: 23 ms


11.079150232267242

In [17]:
#4
from numba import njit
@njit
def for_4(x:np.array):
  m = x.mean()
  return np.absolute(x - m).sum()/m*100/x.shape[0]

def Mape4(rev: pd.DataFrame):
  rating_cl = rev[rev['rating'] != 0]['rating'].to_numpy()
  return for_4(rating_cl)

%time Mape4(reviews)

CPU times: user 782 ms, sys: 7.72 ms, total: 790 ms
Wall time: 801 ms


11.079150232261632

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4